In [3]:
import pandas as pd
import numpy as np
import hdf5plugin
import h5py


IMPORT THE DATA

In [4]:
file = '/Users/maddeerubenson/Downloads/FPV2Mb.h5'
hydro = h5py.File(file,'r')

In [5]:
stage = hydro['hydro/data/channel stage']

In [6]:
# this is for scenario FPV2Mb.h5

start_time = "2016-01-01 00:00:00"
end_time = "2024-01-01 00:00:00"

# Generate the time series with a 5-minute interval
time_series = pd.date_range(start=start_time, end=end_time, freq="5T")

# Create a DataFrame from the time series
df_time_series = pd.DataFrame(time_series, columns=["Datetime"])

df_time_series.head()

/var/folders/45/p22_x33n4s1dbmnzjdjzfr_80000gn/T/ipykernel_83406/2997241107.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_series = pd.date_range(start=start_time, end=end_time, freq="5T")


,Datetime
0,2016-01-01 00:00:00
1,2016-01-01 00:05:00
2,2016-01-01 00:10:00
3,2016-01-01 00:15:00
4,2016-01-01 00:20:00


In [7]:
# get all channel Ids 

channels = hydro['hydro/input/xsect_layer']
channels_df = pd.DataFrame(channels[:], columns = ['chan_no', 'dist', 'elev', 'area', 'width', 'wet_perim'])

all_channels = channels_df['chan_no'].unique()
all_channels

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  65,  66,
        67,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  88,  89,  90,  91,  92,  94,  95,
        96,  97, 101, 102, 104, 105, 106, 107, 108, 111, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 19

CREATE FUNCTION 

In [12]:
def process_and_save_stage_data(stage, df_time_series, channel_ids, output_dir):
    """
    Processes stage data for each channel ID, computes daily summaries,
    and saves the results to CSV files.

    Parameters:
    - stage: numpy array of stage data with shape [timesteps, num_channels, 2] (upstream and downstream)
    - df_time_series: DataFrame containing the Datetime column
    - channel_ids: Array of channel IDs corresponding to the second dimension of the stage array
    - output_dir: Directory path where the CSV files will be saved
    """
    for idx, chan_id in enumerate(channel_ids):
        # Extract upstream and downstream data for the current channel
        upstream = stage[:, idx, 0]
        downstream = stage[:, idx, 1]
        
        # Create DataFrames for upstream and downstream
        df_upstream = pd.DataFrame(upstream, columns=["upstream_stage"])
        df_downstream = pd.DataFrame(downstream, columns=["downstream_stage"])
        
        # Combine with time series data
        df = pd.concat([df_upstream, df_downstream], axis=1)
        all_data = pd.concat([df, df_time_series], axis=1)
        all_data['channel_id'] = chan_id
        
        # Compute combined stage and date
        all_data['combined_stage'] = all_data[['upstream_stage', 'downstream_stage']].mean(axis=1)
        all_data['Date'] = all_data['Datetime'].dt.date
        
        # Compute daily summaries
        summary = (
            all_data.groupby(['Date', 'channel_id'])
            .agg(
                daily_avg=('combined_stage', 'mean'),
                daily_min=('combined_stage', 'min'),
                daily_max=('combined_stage', 'max')
            )
            .reset_index()
        )
        
        # Save to CSV
        output_path = f"{output_dir}/{chan_id}_stage_export.csv"
        summary.to_csv(output_path, index=False)
        print(f"Saved summary for channel {chan_id} to {output_path}")



IMPLEMENT FUNCTION 

In [ ]:
output_dir = '/Users/maddeerubenson/Documents/git/SDG/sdg-dashboard/data-raw/data-exports/exports/fpv2mb/'

# Call the function
process_and_save_stage_data(stage, df_time_series, all_channels[2:], output_dir)

Saved summary for channel 3 to /Users/maddeerubenson/Documents/git/SDG/sdg-dashboard/data-raw/data-exports/exports/fpv2mb//3_stage_export.csv


array([1, 2], dtype=int32)